In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import openslide
import time

In [18]:
import sys
sys.path.append("D:/ProgProjekte/Python/Re-stained_WSIs_Registration/src")

from wsi_registration import TissueDetector, MatcherParameters, WSI_Matcher

In [3]:
# Python < 3.8?
import sys
#! pip install pickle5

if sys.version_info.minor < 8:
    import pickle5 as pickle

In [4]:
import sys
sys.path.append("../../")
from registration_tree import Rect, QuadTree

In [12]:
source_slide_paths = list(Path(r"D:/Datasets/ColonCancer/").glob("*/IHC/*.tif"))
target_slide_paths  = list(Path(r"D:/Datasets/ColonCancer/").glob("*/HE/*.tif"))

source_slide_dict = {name.stem.replace(" X40", "").replace(" HE", "").replace(" IHC", ""): 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_dict.keys()

dict_keys(['CRC-A1-1', 'CRC-A1-10', 'CRC-A1-13', 'CRC-A1-2', 'CRC-A1-5', 'CRC-AI-11 F', 'CRC-AI-14 1G', 'CRC-AI-15', 'CRC-AI-8 E', 'CRC-AI-8 H'])

In [13]:
target_slide_dict = {name.stem.replace(" X40 IHC", "").replace(" X40", "").replace(" HE", ""):  
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_dict.keys()

dict_keys(['CRC-A1-1', 'CRC-A1-10', 'CRC-A1-13', 'CRC-A1-2', 'CRC-A1-5', 'CRC-AI-11 F', 'CRC-AI-14 1G', 'CRC-AI-15', 'CRC-AI-8 E', 'CRC-AI-8 H'])

In [14]:
annotations = pd.read_csv("GT_D240.csv")

annotations["image_name_stem"] = [Path(image_name).stem for image_name in annotations["image_name"]]

annotations["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in annotations["vector"]]
annotations["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in annotations["vector"]]

annotations["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in annotations["vector"]]
annotations["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["center"] = [np.array((center_x, center_y)) for center_x, center_y in zip(annotations["center_x"], annotations["center_y"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,image_name_stem,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26106,"{'x1': 4236, 'x2': 4434, 'y1': 8043, 'y2': 8241}",f52b1e31-3df8-4041-9902-66bc6edef761,21,...,CRC-AI-15 40X,4236,8043,4434,8241,4335.0,8142.0,"[4335.0, 8142.0]",198,198
1,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26107,"{'x1': 6352, 'x2': 6552, 'y1': 23748, 'y2': 23...",b8ca1cea-38ef-4818-b050-d77ef0e44e21,22,...,CRC-AI-15 40X,6352,23748,6552,23948,6452.0,23848.0,"[6452.0, 23848.0]",200,200
2,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26108,"{'x1': 6158, 'x2': 6358, 'y1': 38586, 'y2': 38...",9d817eda-dd12-49b4-a942-9d7ddd77f305,23,...,CRC-AI-15 40X,6158,38586,6358,38786,6258.0,38686.0,"[6258.0, 38686.0]",200,200
3,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26109,"{'x1': 11860, 'x2': 12060, 'y1': 52464, 'y2': ...",04454db4-2e29-4e18-8f8e-271d8e154507,24,...,CRC-AI-15 40X,11860,52464,12060,52664,11960.0,52564.0,"[11960.0, 52564.0]",200,200
4,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26110,"{'x1': 6827, 'x2': 7025, 'y1': 68732, 'y2': 68...",81ee0679-7857-4fc4-a402-df65b0576831,25,...,CRC-AI-15 40X,6827,68732,7025,68934,6926.0,68833.0,"[6926.0, 68833.0]",198,202


In [38]:
%%time

results = []

for name, source in source_slide_dict.items():
    
    if name not in target_slide_dict:
        continue
        
    target = target_slide_dict[name]
        
    source_slide = source["name"]
    target_slide = target["name"]
    
    #patient_id = source_slide.split("-")[2].replace("40X", "X40").split(" X40")[0].replace(" HE", "").replace(".tif", "")
        
    tissue_detector = TissueDetector("LAB_Threshold", threshold=90)
    matcher_parameters = MatcherParameters() 
    
    try:
        start = time.time()
        
        matcher = WSI_Matcher(tissue_detector, matcher_parameters)
        offset = matcher.match(str(source_slide), str(target_slide))
        
        print(f"S: {source_slide.stem}  T: {target_slide.stem} Results: {offset}")
            
        end = time.time()
        
        run_time = end - start
        
        source_name, target_name = source_slide.stem, target_slide.stem
        image_type = "RSC" if "Single" in str(source_slide) else "MSSC"
        
        source_annos = annotations[(annotations["image_name_stem"] == source_name)]
        
        for id, source_anno in source_annos.iterrows():
            
            target_anno = annotations[annotations["image_name_stem"] == target_name].iloc[0]
    
            target_scanner = target_anno.scanner
            image_id = target_anno.image_id
            image_name = target_anno.image_name
            image_width, image_height = target_anno.image_width, target_anno.image_height
        
            new_x1, new_y1 = source_anno.x1 + offset[0], source_anno.y1 + offset[1] 
            new_x2, new_y2 = source_anno.x2 + offset[0], source_anno.y2 + offset[1] 
            
            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}
            
            row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                       vector, source_anno.unique_identifier, source_anno.annotation_type, 
                       source_anno.type_name, f"ReStain", run_time, 1024]
            
            results.append(row)
    except:
         print(f"S: {source_slide.stem}  T: {target_slide.stem} Results: {None, None}")
    
results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", 
                                         "type_name", "method", "runtime", "thumbnail_size"])

results.to_csv(f"ReStain.csv", index=False)


results.head()

S: CRC-A1-1  T: CRC-A1-1 HE Results: (-13471.172090699958, 944.9669558556782)
S: CRC-A1-10  T: CRC-A1-10 HE Results: (None, None)
S: CRC-A1-13  T: CRC-A1-13 HE Results: (-5276.524686858331, 14329.1925674197)
S: CRC-A1-2  T: CRC-A1-2 HE Results: (-326.0891928297093, 37131.597724679275)
S: CRC-A1-5  T: CRC-A1-5 HE Results: (None, None)
S: CRC-AI-11 F X40 IHC  T: CRC-AI-11 F X40 Results: (10475.244186362954, 775.7819065996835)
S: CRC-AI-14 1G X40 IHC  T: CRC-AI-14 1G X40 Results: (420.33765431539916, -1320.1947941054102)
S: CRC-AI-15 X40 IHC  T: CRC-AI-15 X40 Results: (None, None)
S: CRC-AI-8 E X40 IHC  T: CRC-AI-8 E X40 Results: (None, None)
S: CRC-AI-8 H X40 IHC  T: CRC-AI-8 H X40 Results: (12886.831054689581, 2584.0643775818135)
Wall time: 10min 6s


,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method,runtime,thumbnail_size
0,HE,335,CRC-A1-1 HE.tif,MSSC,140203,146077,"{'x1': 12285.827909300042, 'x2': 12485.8279093...",d68e7b9a-d3ae-43ea-9c7c-4dcac427c588,21,L0,ReStain,140.262012,1024
1,HE,335,CRC-A1-1 HE.tif,MSSC,140203,146077,"{'x1': 11899.827909300042, 'x2': 12099.8279093...",0ca22054-a283-4010-bde9-ecb8482c1bd3,22,L1,ReStain,140.262012,1024
2,HE,335,CRC-A1-1 HE.tif,MSSC,140203,146077,"{'x1': 11365.827909300042, 'x2': 11565.8279093...",2a73c263-0dae-4151-82e0-c661ad26930c,24,L3,ReStain,140.262012,1024
3,HE,335,CRC-A1-1 HE.tif,MSSC,140203,146077,"{'x1': 14022.827909300042, 'x2': 14222.8279093...",0b4c38dd-b8a5-49bb-b0be-735b43e47cac,25,L4,ReStain,140.262012,1024
4,HE,335,CRC-A1-1 HE.tif,MSSC,140203,146077,"{'x1': 39035.827909300046, 'x2': 39235.8279093...",325de832-a522-4d95-ab15-d5748eaba0fa,26,L5,ReStain,140.262012,1024


In [ ]:
len()